# 列舉所有資產的riskiness r

In [1]:
import sympy
from pandas import Series,DataFrame
from scipy.optimize import fsolve
import math
import time
import pandas as pd
import numpy  as  np
import os

# 無風險利率
* 我們須扣除無風險利率，得到超額報酬率以計算Riskiness r
* 目前使用LIBOR [1個月利率](https://www.global-rates.com/interest-rates/libor/american-dollar/usd-libor-interest-rate-overnight.aspx)

In [2]:
rfrate = 2.45785 / 12 /100
rfrate

0.0020482083333333333

# 關於超額報酬的數量級:
* 將要帶入公式的數值調整到介在0-100間，目的是為了讓大部分的Riskiness r介在0-100間。

In [3]:
def f1(x,arr_returns):
    return sum( np.exp(-1*arr_returns/x) ) - len(arr_returns)

In [4]:
def get_riskiness_r(guess , arr_returns):
    while (guess<1000):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]

# 計算Riskiness R，並依照R值排序，傳回前十名的ETF

In [10]:
def get_top5_etf(data_file):
    # 讀檔
    #data_file = '../../HW1/ETF_data_set/return_data/Alternatives ETF List (35)/Alternatives ETF List (35)_return.csv';
    df = pd.read_csv(data_file)
    etf_list = list(df.columns)[2:]     #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 因為加上日期所以有修改
    df = df.iloc[:,2:]
    
    # 計算每支ETF的riskiness r
    etf_dic = {}
    for etf_name in etf_list:
        # 過濾平均值正負
        #if df[etf_name].mean(axis=0) < 0:
        #    continue
        arr_returns = df[etf_name] - rfrate
        #arr_returns = arr_returns*1000      #調整數量級
        guess = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic[etf_name] = get_riskiness_r(guess,arr_returns)
    
    # ETF依照riskiness r排序
    sorted_x = sorted(etf_dic.items(), key=lambda kv: kv[1])
        #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前10名
    top5 = sorted_x[:5]
        # top10 = sorted_reversed_x[:10]
    #print(top10)
    
    # 取前10的ETF名稱
    etf_top5= [etf[0] for etf in top5]
    
    #前10的平均
    top5_average = df[etf_top5].mean(axis=1)
    
    #print(etf_top10)
    return etf_top5 , top5_average

# 讀取全部ETF資料夾

In [11]:
data_path = '../../FinalProject/2014to2018RE_data/'
allgroup_top5_return = DataFrame()
allgroup_top5 = dict()
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
        file = subdir + '_return.csv'
        file_path = os.path.join(subdir_path, file)
        #print(file_path)
        if os.path.isfile(file_path):
            # 提出[前10名的名稱,前10名的平均各期報酬]
            [ top5_etf,allgroup_top5_return[subdir] ] = get_top5_etf(file_path)
            print('Group:',subdir)
            print('Top5:',top5_etf)
            print('================')
            allgroup_top5[subdir] = top5_etf
            df = pd.read_csv(file_path)
            date = df['Date']

C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


Group: Alternatives ETF List (35)
Top10: ['MOM', 'SIZ', 'CHEP', 'WTMF', 'QAI']
Group: Broad Asia ETF List (44)
Top10: ['AIA', 'EEMA', 'DBAP', 'AAXJ', 'FPA']
Group: Commodity ETF List (125)
Top10: ['KOLD', 'UCO', 'BOIL', 'DTO', 'SCO']
Group: Consumer Discretionary Equity ETF List (31)
Top10: ['RETL', 'RTH', 'IYC', 'FDIS', 'VCR']
Group: Consumer Staples Equity ETF List (26)
Top10: ['UGE', 'SZK', 'PSL', 'RHS', 'VDC']
Group: Crude Oil ETF List (22)
Top10: ['UCO', 'DTO', 'SCO', 'SZO', 'OLEM']
Group: Currency ETF List (36)
Top10: ['USDU', 'FXE', 'DBV', 'FXA', 'FXS']
Group: Developed Asia Pacific ETF List (108)
Top10: ['IDOG', 'DOO', 'EWT', 'DFJ', 'SCJ']
Group: Developed Markets ETF List 1 (100)
Top10: ['IDOG', 'GII', 'WDIV', 'ACWV', 'IDLV']
Group: Developed Markets ETF List 2 (100)
Top10: ['EFZ', 'DOO', 'DUST', 'RING', 'CIL']
Group: Developed Markets ETF List 3 (93)
Top10: ['CROP', 'IPO', 'PEX', 'FTAG', 'FDTS']
Group: Emerging Asia Pacific ETF List (114)
Top10: ['YINN', 'INDL', 'FXP', 'INCO'

In [15]:
pd.concat([date,allgroup_top5_return],axis=1)

,Date,Alternatives ETF List (35),Broad Asia ETF List (44),Commodity ETF List (125),Consumer Discretionary Equity ETF List (31),Consumer Staples Equity ETF List (26),Crude Oil ETF List (22),Currency ETF List (36),Developed Asia Pacific ETF List (108),Developed Markets ETF List 1 (100),...,Municipal Bond ETF List (29),Preferred Stock ETF List (12),Real Estate ETF List (48),Target Maturity Date Corporate Bond ETF List (24),Technology Equity ETF List (75),Telecom Equity ETF List (10),Total Bond Market ETF List (82),Treasuries ETF List (51),Utilities Equity ETF List (24),Volatility ETF List (18)
0,2014-01-01,-0.001181,-0.062599,-0.009295,-0.084227,-0.020895,-0.001716,-0.012675,-0.028143,-0.028386,...,0.028300,0.046377,-0.002077,0.029821,-0.012968,-0.022846,0.026794,-0.011839,0.035191,0.104873
1,2014-02-01,0.002788,0.045846,-0.042562,0.085938,0.033221,-0.028388,0.012926,0.037423,0.054698,...,0.006666,0.020604,0.014340,0.009802,0.057473,0.000513,0.003333,-0.001868,0.040468,-0.094300
2,2014-03-01,0.002020,0.008330,-0.006595,-0.034762,0.010780,0.007164,0.012210,0.005491,0.014741,...,0.002483,0.016984,-0.007898,0.003390,0.008435,0.023623,0.003039,0.002147,0.034733,-0.026063
3,2014-04-01,0.002650,0.014225,0.003039,-0.024895,-0.003359,0.001472,-0.002046,0.011604,0.030559,...,0.011820,0.019594,0.013188,0.014958,-0.018974,-0.021313,0.011256,-0.000911,0.052254,-0.042291
4,2014-05-01,-0.000359,0.035427,-0.018863,0.016115,0.016677,-0.014523,-0.008007,0.026132,0.015232,...,0.010381,0.012940,0.016519,0.018484,0.039479,0.053959,0.009493,-0.007963,-0.006696,-0.110062
5,2014-06-01,0.005719,0.005484,-0.019271,0.022940,0.009378,-0.014915,0.002942,0.029478,0.004630,...,-0.000718,0.005194,-0.001833,0.003947,0.029538,0.017780,-0.000259,-0.000708,0.044826,-0.094453
6,2014-07-01,-0.011763,0.037926,0.039431,-0.017642,-0.024504,0.039588,-0.011448,-0.008070,-0.012552,...,0.003418,-0.004730,0.000534,0.001233,-0.022618,-0.019899,-0.003366,0.000079,-0.068156,0.066388
7,2014-08-01,0.001929,0.019604,0.000923,0.085675,0.037759,-0.008184,-0.002133,0.010657,0.018035,...,0.011224,0.019842,0.009194,0.015253,0.051575,0.010697,0.014367,-0.007011,0.057956,-0.085349
8,2014-09-01,0.001011,-0.062266,0.005977,-0.022775,-0.009154,0.012628,-0.021874,-0.050441,-0.048581,...,0.001203,-0.012993,-0.024254,-0.009047,-0.018225,-0.032270,-0.005849,0.005484,-0.037208,0.062731
9,2014-10-01,-0.005186,0.021067,0.046918,0.031338,0.029546,0.042463,-0.002852,0.006617,0.019522,...,0.004938,0.015318,0.028411,0.012810,-0.003313,0.016698,0.008017,0.000618,0.095458,-0.081151


In [20]:
def risk(x):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,x)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]
    #return np.std(x)

def sumrisk(weight,all_return):
    w = weight
    a = all_return
    v = 0
    for i in range(len(w)):
        v = v + w[i]*risk(w[i]*(all_return.iloc[:,i]))
    return v

def penalty(w):
    if min(w)<0 or max(w)>1:
        return 100000
    else:
        return 0
def portretmean(weight,all_return):
    w = weight
    a = all_return
    return np.mean(np.dot(np.array(all_return),np.array(weight)))    

def bestweight(w):
    r = []
    r.append(penalty(w)+np.abs(sum(w)-1))
    for i in range(len(w)-1):
        v =  np.abs( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) ) \
            + np.abs( risk(w[i+1]*(all_return.iloc[:,i+1]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i+1])-rfrate)/(portretmean(w,all_return)-rfrate))
        r.append(v) 
    print('權重:',w,'誤差值:',v+penalty(w)+np.abs(sum(w)-1))
    return r

In [21]:
def fourdrop(x):
    a = []
    for i in range(len(x)):
        a.append(round(sum(x[i]),3))
    return a
x = np.array(allgroup_top10_return.iloc[:,[0]])
x = fourdrop(x)
y = np.array(allgroup_top10_return.iloc[:,[1]])
y = fourdrop(y)
all_return = pd.DataFrame([x,y]).T
all_return = all_return

In [22]:
x = np.array([-0.01,0,0.0258])
y = np.array([-0.05,0,0.134])
all_return = pd.DataFrame([x,y]).T

In [32]:
all_return = allgroup_top10_return.iloc[:,list(range(0, 32))]
n = all_return.shape[1]
[1/n]*n

[0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125,
 0.03125]

In [33]:
a = list(range(0, 2))
all_return = allgroup_top10_return.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)
C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125

權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938272427682
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938273561566
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738560733
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0

權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382737570457
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938275550922
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738476218
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 

In [37]:
a = list(range(0, 5))
all_return = allgroup_top5_return.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 19.453036746318976
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 19.453036746318976
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 19.453036746318976
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 19.453036737663062
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 19.45303639826295
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 19.45303659221885
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 19.453036462202626
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 19.45303650025005
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 19.453036745871863
權重: [0.14285714 0.14285714 0.14